In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os

from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
frame = pd.read_csv("/data/shruti/ONR/small_data/facebook_linkpost_text.csv", header=0)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data=frame

In [4]:
data = data.dropna(subset=['message','handle'])
data = data.drop_duplicates(subset=['handle','message'])
data['is_duplicated'] = data.duplicated(subset=['message'], keep=False)
data.head()

,Unnamed: 0,date,title,caption,description,message,link,postUrl,score,handle,platformId,id,is_duplicated
0,0,2021-05-07 15:18:16,औरंगाबाद जिल्ह्यातील रिक्त पदे भरण्याची प्रक्र...,navarashtra.com,सदर याचिकेची पहिली सुनावणी दिनांक ०४ मे २०२१ र...,https://www.navarashtra.com/aurangabad-news-ma...,https://www.navarashtra.com/aurangabad-news-ma...,https://www.facebook.com/719366294805607/posts...,-1.546592,imtiaz.jaleel,1.000444e+14,6611956.0,False
1,1,2021-05-07 05:36:34,"खाजगी रुग्णालयांमधील लाखोंची लूट थांबवा, माण -...",navarashtra.com,सध्याचे वास्तव पहाता खाजगी रुग्णालये रुग्णाला ...,"खाजगी रुग्णालयांमधील लाखोंची लूट थांबवा, माण –...",https://www.navarashtra.com/satara-news-marath...,https://www.facebook.com/104963460865558/posts...,0.000000,JaykumarGoreSpeaks,1.049635e+14,17548944.0,False
28,28,2021-07-01 11:15:25,Watch how Hema Malini screamed like a kid on s...,hindustantimes.com,NaN,This video is proof of the sweetest bond betwe...,https://www.hindustantimes.com/entertainment/b...,https://www.facebook.com/111445058579/posts/10...,75.561947,hindustantimes,1.114451e+11,33887.0,False
29,29,2021-07-01 01:43:47,"Steve Waugh beats Virat Kohli, Ricky Ponting, ...",hindustantimes.com,"Steve Waugh, who captained Australia in 57 Tes...","Humbled, thank you...",https://www.hindustantimes.com/cricket/steve-w...,https://www.facebook.com/100044384051423/posts...,12.641566,SteveWaughOfficial,1.000444e+14,1167655.0,False
30,30,2021-07-01 13:28:44,NaN,NaN,NaN,RT @Dharma2X: India may procure Israeli anti-d...,https://www.facebook.com/Swamy39/photos/a.2058...,https://www.facebook.com/1577113465933531/post...,1.889492,Swamy39,1.577113e+15,1040416.0,False


In [5]:
dup_data = data.loc[data['is_duplicated']==True]

In [6]:
from random import sample
unique_texts = list(set(dup_data['message'].tolist()))
print(len(unique_texts))
sample_unique = sample(unique_texts, 10000)

sdata = dup_data.loc[dup_data['message'].isin(sample_unique)]

434038


In [7]:
def assign_unique_group(text):
    uid = sample_unique.index(text)
    return uid

In [8]:
sdata['text_id'] = sdata['message'].parallel_apply(lambda x: assign_unique_group(x))

<ipython-input-8-3e12118ee26f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdata['text_id'] = sdata['message'].parallel_apply(lambda x: assign_unique_group(x))


In [30]:
connections = sdata.groupby('text_id')['handle'].apply(list).reset_index()
connections.head()

,text_id,handle
0,0,"[banglahuntofficial, banglahunt, indianews.ben..."
1,1,"[socialnewsxyz, agkfire]"
2,2,"[zeebusinessonline, ZeeBusinessHindi]"
3,3,"[socialnewsxyz, agkfire]"
4,4,"[KanakNews, kanaknewsroom]"


In [31]:
nodes = defaultdict(int)
edges = defaultdict(int)

In [32]:
for idx, row in connections.iterrows():
    screenlist = row['handle']
    for i in range(len(screenlist)):
        nodes[screenlist[i]]+=1
        
    for i in range(len(screenlist)-1):
        for j in range(i+1,len(screenlist)):
            stuple = sorted([screenlist[i], screenlist[j]])
            edges[(stuple[0], stuple[1])]+=1

In [33]:
import networkx as nx

In [34]:
G = nx.Graph()
for e in edges:
    if edges[e] > 2:
        if e[0] not in G.nodes:
            G.add_node(e[0], size=nodes[e[0]])
        if e[1] not in G.nodes:
            G.add_node(e[1], size=nodes[e[1]])
            
        G.add_edge(e[0], e[1], weight=edges[e])


In [35]:
nx.write_gexf(G, "../lite_data/astroturf_facegroups_short.gexf", encoding='utf-8')

In [37]:
sdata.sort_values(by=['title']).head(50)

,Unnamed: 0,date,title,caption,description,message,link,postUrl,score,handle,platformId,id,is_duplicated,text_id
18444097,11608540,2021-06-08 07:53:45,"""100 ஆண்டு சமூகநீதிப் பயணத்தில் ஓர் மைல் கல்""-...",vikatan.com,"10 ஆண்டுகளுக்கும் மேலாக வீதிகளிலும், நீதிமன்றத...",அனைத்து சாதி மாணவர்கள் சங்க தலைவர் ரங்கநாதன் ப...,https://www.vikatan.com/government-and-politic...,https://www.facebook.com/264721000232285/posts...,-7.250000,JuniorVikatan,2.647210e+14,840790.0,True,7939
18444029,11608472,2021-06-08 07:53:45,"""100 ஆண்டு சமூகநீதிப் பயணத்தில் ஓர் மைல் கல்""-...",vikatan.com,"10 ஆண்டுகளுக்கும் மேலாக வீதிகளிலும், நீதிமன்றத...",அனைத்து சாதி மாணவர்கள் சங்க தலைவர் ரங்கநாதன் ப...,https://www.vikatan.com/government-and-politic...,https://www.facebook.com/189960617729403/posts...,-1.081081,vikatanweb,1.899606e+14,329420.0,True,7939
5762148,4009120,2021-08-18 14:44:38,"""3 தடவை பூமி பூஜை போட்டாச்சு; மயானத்திற்குச் ச...",vikatan.com,"ஒவ்வொரு இறப்பின் போதும், சுமார் 20 ஆயிரம் செலவ...","""3 தடவை பூமி பூஜை போட்டாச்சு; மயானத்திற்குச் ச...",https://www.vikatan.com/news/tamilnadu/saanarb...,https://www.facebook.com/264721000232285/posts...,-7.000000,JuniorVikatan,2.647210e+14,840790.0,True,9484
5762086,4009058,2021-08-18 14:44:38,"""3 தடவை பூமி பூஜை போட்டாச்சு; மயானத்திற்குச் ச...",vikatan.com,"ஒவ்வொரு இறப்பின் போதும், சுமார் 20 ஆயிரம் செலவ...","""3 தடவை பூமி பூஜை போட்டாச்சு; மயானத்திற்குச் ச...",https://www.vikatan.com/news/tamilnadu/saanarb...,https://www.facebook.com/189960617729403/posts...,-3.187500,vikatanweb,1.899606e+14,329420.0,True,9484
18115509,11279952,2021-07-26 06:31:20,"""He Didn't Act"": Shekhar Kapur On Naseeruddin ...",ndtv.com,"""Naseer's performance was not just a performan...","""Naseer's performance was not just a performan...",https://www.ndtv.com/entertainment/he-didnt-ac...,https://www.facebook.com/102527030797/posts/10...,29.379747,ndtv,1.025270e+11,32209.0,True,7752
18115777,11280220,2021-07-26 06:32:22,"""He Didn't Act"": Shekhar Kapur On Naseeruddin ...",ndtv.com,"""Naseer's performance was not just a performan...","""Naseer's performance was not just a performan...",https://www.ndtv.com/entertainment/he-didnt-ac...,https://www.facebook.com/98462422918/posts/101...,-0.846154,ndtvmovies,9.846242e+10,386162.0,True,7752
4287822,2534794,2021-04-21 04:55:34,"""How?"": Son Harshvardhan And Other Celebs Hila...",ndtv.com,"""Anil, 18+ starts from May 1! How did they giv...","""Anil, 18+ starts from May 1! How did they giv...",https://www.ndtv.com/entertainment/how-son-har...,https://www.facebook.com/102527030797/posts/10...,-3.934426,ndtv,1.025270e+11,32209.0,True,9756
4288204,2535176,2021-04-21 04:55:16,"""How?"": Son Harshvardhan And Other Celebs Hila...",ndtv.com,"""Anil, 18+ starts from May 1! How did they giv...","""Anil, 18+ starts from May 1! How did they giv...",https://www.ndtv.com/entertainment/how-son-har...,https://www.facebook.com/98462422918/posts/101...,-40.000000,ndtvmovies,9.846242e+10,386162.0,True,9756
18038173,11202616,2021-12-01 03:00:10,"""I'm Done, Goodbye..."": Comic After 12 Shows C...",ndtv.com,Comedian Munawar Faruqui indicated he may not ...,"""I'm Done, Goodbye..."": Comic After 12 Shows C...",https://bit.ly/3roRrDs,https://www.facebook.com/130148817071986/posts...,-4.100000,AtheistRepublic,1.301488e+14,327000.0,True,1472
18038298,11202741,2021-12-01 03:00:10,"""I'm Done, Goodbye..."": Comic After 12 Shows C...",ndtv.com,Comedian Munawar Faruqui indicated he may not ...,"""I'm Done, Goodbye..."": Comic After 12 Shows C...",https://bit.ly/3roRrDs,https://www.facebook.com/26056363103/posts/101...,-26.000000,GodorNoGod,2.605636e+10,1121621.0,True,1472
